In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna
import optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 8.0 MB/s 
     |████████████████████████████████| 81 kB 12.4 MB/s 
     |████████████████████████████████| 209 kB 74.1 MB/s 
     |████████████████████████████████| 78 kB 10.4 MB/s 
     |████████████████████████████████| 49 kB 7.9 MB/s 
     |████████████████████████████████| 147 kB 69.3 MB/s 
     |████████████████████████████████| 112 kB 67.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=c46084ed0b7833055b3e61e2c3c03f3bc6ebb6261089b1790e8cc056e782c8f4
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
cd /content/drive/MyDrive/수방이작업실/

/content/drive/.shortcut-targets-by-id/1x3tcgMmudxP_hPBhIjW1UdtxSUPqaxa_/수방이작업실


In [4]:
import numpy as np 
import pandas as pd
from time import time 
import pprint

import warnings
warnings.filterwarnings('ignore')


import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [5]:
data = pd.read_csv('./solar_slide_timeseriesed.csv')
data_x = data[data['datetime'] < '2022-05-01']
data_y = data_x['target']
data_x = data_x.iloc[:,2:]

test_x = data[(data['datetime'] >= '2022-05-01') & (data['datetime'] < '2022-07-01')]
test_x.drop(['datetime','target'], axis=1, inplace=True)

In [6]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(data_x, data_y, test_size=0.1, random_state=1234, shuffle=True)

In [7]:
from sklearn.preprocessing import  MinMaxScaler

mm_scaler = MinMaxScaler()
mm_scaler.fit(train_x)
train_x_scaled = mm_scaler.transform(train_x)
val_x_scaled = mm_scaler.transform(val_x)
test_x_scaled = mm_scaler.transform(test_x)

train_y = np.array(train_y)
val_y = np.array(val_y)

In [8]:
def NMAE(true, pred, nominal=1100):
    absolute_error = np.abs(true - pred)
    absolute_error /= nominal
    target_idx = np.where(true >= nominal*0.1)
    return 100 * np.array([absolute_error[i] for i in target_idx]).mean()

In [9]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
def objective(trial,data=train_x_scaled,target=train_y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.1,random_state=1234)
    param = {
  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12,13,14,15]),
        'random_state': trial.suggest_categorical('random_state', [1234]),
        'min_child_weight': trial.suggest_int('min_child_weight', 100, 1000),

        'objective':'reg:squarederror',
        'tree_method':'gpu_hist',
    }
    model = XGBRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=10,verbose=False)
    
    preds = model.predict(test_x)
    
    nmae = NMAE(test_y, preds, 1100)
    
    return nmae

In [11]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-08-22 06:51:44,939] A new study created in memory with name: no-name-200e899a-eee0-4ec6-8dca-e995e4af35ea
[I 2022-08-22 06:52:19,534] Trial 0 finished with value: 0.9336654370202057 and parameters: {'lambda': 0.008979089171689974, 'alpha': 0.0032345686929993816, 'colsample_bytree': 0.6, 'subsample': 0.6, 'learning_rate': 0.01, 'max_depth': 9, 'random_state': 1234, 'min_child_weight': 926}. Best is trial 0 with value: 0.9336654370202057.
[I 2022-08-22 06:52:56,209] Trial 1 finished with value: 0.486965899211267 and parameters: {'lambda': 0.003949424327610034, 'alpha': 4.772874131473659, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.018, 'max_depth': 15, 'random_state': 1234, 'min_child_weight': 739}. Best is trial 1 with value: 0.486965899211267.
[I 2022-08-22 06:53:12,845] Trial 2 finished with value: 0.9109356825978524 and parameters: {'lambda': 0.009911628737925402, 'alpha': 3.4719526459316175, 'colsample_bytree': 0.3, 'subsample': 0.7, 'learning_rate': 0.018,

Number of finished trials: 50
Best trial: {'lambda': 0.7068656374537707, 'alpha': 0.00408705671730144, 'colsample_bytree': 1.0, 'subsample': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'random_state': 1234, 'min_child_weight': 154}


In [ ]:
params = study.best_params
params['random_state'] = 1234
params['n_estimators'] = 1000
params['metrics'] = 'rmse'

In [ ]:
params['cat_smooth'] = params.pop('min_data_per_groups')

KeyError: ignored

In [ ]:
params